In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.preprocessing import MinMaxScaler
import math
import pickle


In [2]:
df=pd.read_csv("rslot.csv")
df

,consignment_id,recipient_id,sender_id,sender_name,recipient_name,item_type,recipient_phone_number,sender_phone_number,initial_time_schedule,selected_time_schedule,recipient_address,scheduled_delivery_date,actual_delivery_date,delivery_attempts,delivery_outcome,reason_for_failure,past_delivery_success,pincode,timestamp_of_booking,delivery_time_modification
0,IN3456789012345,274,452,Aaditya Patel,Nikita Singh,Document,907760524845,910876543211,09:45,09:45,"42, 1st Floor, 7th Main Road, Indiranagar, Ben...",2023-04-11,2023-04-16,2.0,Successful,NaN,0.95,341636,2023-04-23 15:30:00,No
1,IN34567890123456,40,3589,Rajesh Patel,Nikita Singh,Package,917060248204,1032546007942,15:00,15:00,"25, 2nd Floor, 10th Main Road, Indiranagar, Be...",2023-03-28,2023-04-06,0.0,Pending,NaN,0.76,508320,2023-04-15 09:00:00,No
2,IN6789012345678,72,2075,Priya Singh,Aarav Tandon,Document,919987654323,1054233548861,09:15,09:15,"345, 5th Street, Chennai, Tamil Nadu 600002",2023-04-19,2023-04-15,2.0,Successful,NaN,0.50,170105,2023-05-08 15:00:00,No
3,IN567890123ABCDEF5678,11,123,Suresh Verma,Ankit Patel,Parcel,919987654323,1298279242771,09:30,15:30,"67, 2nd Cross, 4th Main, Malleswaram, Bengalur...",2023-04-10,2023-04-10,1.0,Successful,NaN,0.82,700065,2023-03-28 08:00,No
4,IN456789012345,219,572,Sita Devi,Meena Patel,Document,870680816194,910876543211,11:00,17:00,"Plot No. 34, Sector 12, Noida, Uttar Pradesh, ...",2023-05-23,2023-05-02,2.0,Successful,NaN,0.88,700065,2023-03-29 14:00:00,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,IN3456789012345,398,123,Nikhil Patel,Mira Kapoor,Document,919987654323,1099720975310,10:00,11:45,"Plot No. 30, Fifth Cross Road, Malleswaram, Be...",2023-04-11,2023-03-15,2.0,Successful,NaN,0.73,255568,2023-04-19 09:30:00,No
996,IN567890123456,11,1007,Ram Mohan,Rita Singh,Parcel,919987654323,910876543211,10:00,15:30,"House No. 12, Street B, New Town, Kolkata, Wes...",2023-05-19,2023-05-19,1.0,Failed,Recipient Not Available,0.93,110001,2023-03-28 15:30:00,No
997,IN678901234567,324,1131,Ram Singh,Rahul Kumar,Parcel,919987654323,910876543211,11:00,10:30,"98, 1st Cross, 1st Main, Koramangala, Bengalur...",2023-04-10,2023-04-18,0.0,Pending,NaN,NaN,110001,2023-04-19 09:30:00,Yes
998,IN1234567890123,196,955,Sangeeta Gupta,Divya Sharma,Document,889201971001,910876543211,11:30,15:30,"Kalyan Estate, Juhu, Mumbai, Maharashtra, Indi...",2023-07-19,2023-04-08,0.0,Pending,NaN,0.95,510394,2023-04-18 09:00:00,No


In [3]:
df['delivery_time_modification']=df['delivery_time_modification'].replace("Yes","yes")
df['delivery_time_modification']=df['delivery_time_modification'].replace("No","no")

In [4]:
df['scheduled_delivery_date'] = pd.to_datetime(df['scheduled_delivery_date'])
df['actual_delivery_date'] = pd.to_datetime(df['actual_delivery_date'])
df['timestamp_of_booking'] = pd.to_datetime(df['timestamp_of_booking'])

In [5]:
def time_to_minutes(time_str):
    if pd.isna(time_str):
        return None
    h, m = map(int, time_str.split(':'))
    return h * 60 + m

df['selected_time_minutes'] = df['selected_time_schedule'].apply(time_to_minutes)

In [6]:
df['delivery_attempts'] = df['delivery_attempts'].fillna(1)
df['reason_for_failure']=df['reason_for_failure'].fillna("NotApplicable") 
df['past_delivery_success'] = df['past_delivery_success'].fillna(-1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   consignment_id              1000 non-null   object        
 1   recipient_id                1000 non-null   int64         
 2   sender_id                   1000 non-null   int64         
 3   sender_name                 1000 non-null   object        
 4   recipient_name              1000 non-null   object        
 5   item_type                   1000 non-null   object        
 6   recipient_phone_number      1000 non-null   int64         
 7   sender_phone_number         1000 non-null   int64         
 8   initial_time_schedule       1000 non-null   object        
 9   selected_time_schedule      1000 non-null   object        
 10  recipient_address           1000 non-null   object        
 11  scheduled_delivery_date     1000 non-null   datetime64[ns

In [8]:
scaler=MinMaxScaler()
df['time_preference']=scaler.fit_transform(df[['selected_time_minutes']])

In [9]:
data = df[['recipient_id', 'sender_id', 'time_preference']]
print(data)

     recipient_id  sender_id  time_preference
0             274        452            0.075
1              40       3589            0.600
2              72       2075            0.025
3              11        123            0.650
4             219        572            0.800
..            ...        ...              ...
995           398        123            0.275
996            11       1007            0.650
997           324       1131            0.150
998           196        955            0.650
999           329       1246            0.300

[1000 rows x 3 columns]


In [10]:
# Create a Reader object with appropriate rating_scale
reader = Reader(rating_scale=(0, 1))



In [11]:
# Load the data into Surprise's format
data = Dataset.load_from_df(data[['recipient_id', 'sender_id', 'time_preference']], reader)



In [12]:
# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)



In [13]:
# Use Singular Value Decomposition (SVD) for matrix factorization
algo = SVD()



In [14]:
# Train the algorithm on the trainset
algo.fit(trainset)

# Predict on the testset
predictions = algo.test(testset)



In [15]:
# Calculate accuracy metrics
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")



RMSE: 0.2471
RMSE: 0.24714818556953247


In [16]:
# Example of predicting time preference for a specific recipient and sender
# recipient_id = 274
# sender_id = 452
# predicted_time_pref = algo.predict(recipient_id, sender_id).est
# print(f"Predicted time preference: {predicted_time_pref}")

In [18]:
# converting into the pickle file

with open("recipient.pkl", 'wb')as file:
    pickle.dump(algo,file)